In [11]:
import numpy as np
import pandas as pd
from scipy import interpolate

# Plot Sound Speed Profile

## Read Files

In [8]:
ssp_basu_path = '../SoundSpeed/basu_09.txt'
column_names = [" r/R_sun", "c [cm/s]", "Sigma_c"]
ssp_obs = pd.read_csv(ssp_basu_path, delim_whitespace=True, skiprows=8, names=column_names)
#print(ssp_obs)

In [10]:
data_gs98_path = "../solar_23.05.1/solar.profile"
skiprows = 6
column_names = ["radius", "csound"]

model_gs98 = pd.read_csv(data_gs98_path, delim_whitespace=True, skiprows=skiprows, usecols=[9, 10], names=column_names)
#print(model_gs98)

## Interpolate the observed data
Since the observed data will be used to compare both models